In [1]:
in_funds = ['VFINX', 'PRIDX']
out_funds = 'VUSTX'
benchmark = 'VFINX'
risk_free_fund = '^IRX'

In [2]:
import yfinance as yf

# Download historical data
data = {}
all_funds = in_funds + [out_funds, risk_free_fund]
for symbol in all_funds:
    print(symbol)
    yf_ticker = yf.Ticker(symbol)
    yf_data = yf_ticker.history(period='max')
    data[symbol] = yf_data
    
benchmark = yf.Ticker(benchmark).history(period='max')

VFINX
PRIDX
VUSTX
^IRX


In [18]:
# Select only month-end prices
monthend = {}
for symbol, df in data.items():
    day1 = list(df.index.month)
    day2 = list(df.index.month)
    day2.pop(0)
    day2.append(-1)
    days = zip(day1, day2)
    truth_vals = [False if x[0] == x[1] else True for x in days]
    df_month_end = df[truth_vals].copy()
    df_month_end.index = df_month_end.index.strftime('%Y-%m')
    monthend[symbol] = df_month_end

In [20]:
# Calculate momentum signals
import numpy

risk_free_rate = monthend[risk_free_fund]['Close'] / 12

# 1-month momentum
for symbol, df in monthend.items():
    close = numpy.array(df['Close'])
    momentum1 = (((close[1:]/close[:-1])-1)*100)
    momentum1 = numpy.insert(momentum1, 0, 0)
    df['Momentum1'] = momentum1
    df['Momentum1'] = df['Momentum1'] - risk_free_rate

# 3-month momentum
for symbol, df in monthend.items():
    close = numpy.array(df['Close'])
    momentum3 = (((close[3:]/close[:-3])-1)*100)
    momentum3 = numpy.insert(momentum3, 0, [0, 0, 0])
    df['Momentum3'] = momentum3
    df['Momentum3'] = df['Momentum3'] - (risk_free_rate.rolling(3).sum())

# 6-month momentum
for symbol, df in monthend.items():
    close = numpy.array(df['Close'])
    momentum6 = (((close[6:]/close[:-6])-1)*100)
    momentum6 = numpy.insert(momentum6, 0, [0, 0, 0, 0, 0, 0])
    df['Momentum6'] = momentum6
    df['Momentum6'] = df['Momentum6'] - (risk_free_rate.rolling(6).sum())

# ADM Signal
for symbol, df in monthend.items():
    momentums = numpy.array((df['Momentum1'], df['Momentum3'], df['Momentum6']))
    df['ADMSignal'] = numpy.average(momentums, axis=0)

In [53]:
# Align time-axis for all symbols
import pandas
merged = None
for symbol, df in monthend.items():
    if not symbol in in_funds:
        continue
    if merged is None:
        merged = pandas.DataFrame(df['ADMSignal'])
        merged[f'{symbol}-ADMSignal'] = merged['ADMSignal']
        merged = merged.drop(columns=['ADMSignal'])
    else:
        df2 = pandas.DataFrame(df['ADMSignal'])
        df2[f'{symbol}-ADMSignal'] = df2['ADMSignal']
        df2 = df2.drop(columns=['ADMSignal'])
        merged = merged.merge(df2, how='inner', on='Date')
time_aligned = merged
time_aligned.tail(6)

,VFINX-ADMSignal,PRIDX-ADMSignal
Date,,
2020-04,-0.128895,0.319519
2020-05,1.890366,6.678406
2020-06,6.325886,13.070038
2020-07,6.862086,13.719571
2020-08,14.250359,17.870925
2020-09,10.645134,20.722695


In [73]:
# Build portfolio
symbols = pandas.DataFrame(time_aligned.idxmax(axis=1), columns=['Symbol'])
symbols['Symbol'] = [x.split('-')[0] for x in symbols['Symbol']]
signal  = pandas.DataFrame(time_aligned.max(axis=1), columns=['Signal'])
in_signal = symbols.merge(signal, how='inner', on='Date')
for idx in in_signal[in_signal['Signal'] < 0].index:
    in_signal.loc[idx, 'Symbol'] = out_funds
portfolio = in_signal
portfolio.tail(6)

,Symbol,Signal
Date,,
2020-04,PRIDX,0.319519
2020-05,PRIDX,6.678406
2020-06,PRIDX,13.070038
2020-07,PRIDX,13.719571
2020-08,PRIDX,17.870925
2020-09,PRIDX,20.722695


In [74]:
# Compute portfolio returns & growth of 10k
current_asset = None
cash = 10000
qty = 0
for date in portfolio.index:
    new_asset = portfolio.loc[date]['Symbol']
    if current_asset is not None:
        # sell the current asset
        new_cash = monthend[current_asset].loc[date]['Close'] * qty
        qty = 0
        monthly_return = new_cash / cash - 1
        cash = new_cash
    else:
        monthly_return = 0.0
    current_asset = new_asset
    qty = cash / monthend[new_asset].loc[date]['Close']
    portfolio.loc[date, 'MonthlyReturn'] = monthly_return
    portfolio.loc[date, 'Value'] = cash
    portfolio.loc[date, 'Quantity'] = qty
    portfolio.loc[date, 'Price'] = monthend[new_asset].loc[date]['Close']

In [75]:
# Update dates so they reflect the month that the asset should be owned
import pandas
dates = []
for dt in portfolio.index:
    yr, mo = map(int, dt.split('-'))
    if mo == 12:
        yr += 1
        mo = 1
    else:
        mo += 1
    dates.append((yr, mo))
portfolio.index = pandas.MultiIndex.from_tuples(dates, names=('Year', 'Month'))

In [78]:
from datetime import datetime

#  Compute daily returns for portfolio
returns = []

# Step 1 get all trading days for the period that the portfolio is active
start_yr, start_mo = list(portfolio.index)[0]
start_date = datetime(start_yr, start_mo, 1)
end_yr, end_mo = list(portfolio.index)[-1]
end_date = datetime(end_yr, end_mo, 1)

trading_days = []
for symbol, df in data.items():
    if list(df.index)[0] < start_date:
        for dt in list(df.index):
            if dt >= start_date and dt <= end_date:
                trading_days.append(dt)
        break
        
# Step 2 for each trading day compute portfolio return
last = trading_days[0]
asset = portfolio.loc[(last.year, last.month)]['Symbol']
for day in trading_days[1:]:
    current_close = data[asset].loc[day]['Close']
    previous_close = data[asset].loc[last]['Close']
    ret = current_close / previous_close - 1
    returns.append({'Date': day, 'Return': ret})

    # check if asset was sold
    month = (day.year, day.month)
    if month in list(portfolio.index):
        asset = portfolio.loc[month]['Symbol']
    
    # update the last trading day
    last = day

returns_df = pandas.DataFrame([x['Return'] for x in returns], index=pandas.DatetimeIndex([x['Date'] for x in returns], name='Date'), columns=['Returns'])
returns_df.tail()

,Returns
Date,
2020-09-23,-0.008414
2020-09-24,-0.003040
2020-09-25,0.009909
2020-09-28,0.012579
2020-09-29,0.002236


In [79]:
# Print out last few items, to show what should be purchased
portfolio.tail(24)

Symbol     Signal  MonthlyReturn         Value       Quantity  \
Year Month                                                                 
2018 11     VUSTX  -2.861458      -0.068523  2.017244e+06  192485.088621   
     12     VUSTX  -0.424034       0.018130  2.053816e+06  192485.088621   
2019 1      VUSTX -10.466017       0.055295  2.167382e+06  192485.088621   
     2      VFINX   1.084642       0.004440  2.177006e+06    8957.030867   
     3      VUSTX  -0.158693       0.032010  2.246692e+06  201316.492155   
     4      VFINX   3.937400       0.054659  2.369495e+06    9267.424565   
     5      VFINX   7.073659       0.040402  2.465228e+06    9267.424565   
     6      VUSTX  -1.117796      -0.063644  2.308330e+06  187364.457024   
     7      VFINX   9.299837       0.010552  2.332687e+06    8749.765529   
     8      VFINX   4.169916       0.014254  2.365937e+06    8749.765529   
     9      VFINX   3.195844      -0.015902  2.328313e+06    8749.765529   
     10     VFINX   2.626597       0.018602  2.371624e+06    8749.765529   
     11     VFINX   2.374435       0.021546  2.422723e+06    8749.765529   
     12     VFINX   8.400887       0.036224  2.510483e+06    8749.765529   
2020 1      PRIDX   7.468355       0.030078  2.585993e+06   37845.649085   
     2      VFINX   4.852397      -0.017123  2.541714e+06    8604.312094   
     3      VUSTX  -0.380001      -0.082431  2.332199e+06  158760.979788   
     4      VUSTX -15.102111       0.056501  2.463970e+06  158760.979788   
     5      PRIDX   0.319519       0.018686  2.510011e+06   40746.933286   
     6      PRIDX   6.678406       0.105195  2.774051e+06   40746.933286   
     7      PRIDX  13.070038       0.042303  2.891402e+06   40746.933286   
     8      PRIDX  13.719571       0.063839  3.075986e+06   40746.933286   
     9      PRIDX  17.870925       0.078289  3.316800e+06   40746.933286   
     10     PRIDX  20.722695      -0.008845  3.287463e+06   40746.933286   

             Price  
Year Month          
2018 11      10.48  
     12      10.67  
2019 1       11.26  
     2      243.05  
     3       11.16  
     4      255.68  
     5      266.01  
     6       12.32  
     7      266.60  
     8      270.40  
     9      266.10  
     10     271.05  
     11     276.89  
     12     286.92  
2020 1       68.33  
     2      295.40  
     3       14.69  
     4       15.52  
     5       61.60  
     6       68.08  
     7       70.96  
     8       75.49  
     9       81.40  
     10      80.68

In [80]:
# Compute benchmark returns
close = numpy.array(benchmark['Close'])
rets = (close[1:]/close[:-1]) - 1
rets = numpy.insert(rets, 0, 0)
benchmark['Returns'] = rets
benchmark_aligned = benchmark[benchmark.index > start_date]

In [81]:
import pyfolio
pyfolio.create_full_tear_sheet(returns_df['Returns'], benchmark_rets=benchmark_aligned['Returns'])

Start date,1989-01-04
End date,2020-09-29
Total months,380
,Backtest
Annual return,19.8%
Cumulative returns,30782.7%
Annual volatility,13.7%
Sharpe ratio,1.39
Calmar ratio,0.64
Stability,0.98
Max drawdown,-31.0%


AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'